In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

with open('text.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())

data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount) #преобразуем в набор векторов

model = Sequential()
model.add(Embedding(maxWordsCount, #число слов в словаре
                    256, #число выходов в полносвязном Embedding-слое
                    input_length = inp_words)) #размер входного вектора (по которым будет строится прогноз)
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)


def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]

    return res


res = buildPhrase("жизнь слишком коротка")
print(res)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 3, 256)            256000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               49280     
                                                                 
 dense_8 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 434,280
Trainable params: 434,280
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
52/52 [==============================] - 1s 6ms/step - loss: 6.8389 - accuracy: 0.0385
Epoch 2/50
52/52 [==============================] - 0s 8ms/step - loss: 6.1364 - accuracy: 0.0559
Epoch 3/50
52/52 [==============================] - 0s 6ms/step - loss: 5.7678 - accuracy: 0.0787
Epoch 4/50
52/52 [